In [1]:
# Knowledge Base >> https://github.com/iftymahmud/UWindsor_KnowledgeBase

In [2]:
!pip install openai==1.10.0

!pip install llama-index==0.9.42.post1

!pip install gpt-index==0.4.24

!pip install langchain==0.0.148
!pip install Python==3.11.7


ERROR: Could not find a version that satisfies the requirement Python==3.11.7 (from versions: none)
ERROR: No matching distribution found for Python==3.11.7


In [3]:
from llama_index import SimpleDirectoryReader, GPTListIndex, readers, GPTVectorStoreIndex, get_response_synthesizer, StorageContext, LLMPredictor, PromptHelper, ServiceContext
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SimilarityPostprocessor
from langchain.chat_models import ChatOpenAI
import sys
import os
from IPython.display import Markdown, display

def construct_index(directory_path):

    max_input_size = 4096
    num_output = 2000
    chunk_size_limit = 600

    prompt_helper = PromptHelper(max_input_size, num_output, chunk_overlap_ratio= 0.1, chunk_size_limit=chunk_size_limit)
    
    llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0.5, model_name="gpt-3.5-turbo-instruct", max_tokens=num_output))

    documents = SimpleDirectoryReader(directory_path).load_data()

    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    return index


def ask_ai(index):
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=10,
    )

    response_synthesizer = get_response_synthesizer()

    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer,
        node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
    )

    while True:
        response = query_engine.query(input("<< What do you want to ask? >> \n"))
        display(Markdown(f"Response: <b>{response.response}</b>"))

In [4]:
os.environ["OPENAI_API_KEY"] = 'YOUR API KEY HERE'

In [5]:
index = construct_index("data")

LLMPredictor is deprecated, please use LLM instead.


In [ ]:
ask_ai(index)

<< What do you want to ask? >> 
Which courses do I need to do in the MAC Program?


Response: <b>You need to take the following 8 courses in the MAC Program:
- COMP-8117: Advanced Software Engineering Topics
- COMP-8157: Advanced Database Topics
- COMP-8347: Internet Applications and Distributed Systems
- COMP-8547: Advanced Computing Concepts
- COMP-8567: Advanced Systems Programming
- COMP-8677: Networking and Data Security
- COMP-8967: Internship/Project I
- COMP-8977: Internship/Project II</b>